In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [ ]:
pd.set_option('display.max_rows', None)

## Importing data
### Link to the challenge: 
 https://hecktor.grand-challenge.org/Data/
 
 https://hecktor.grand-challenge.org/Evaluation/

## Leaderboard:
### https://hecktor.grand-challenge.org/evaluation/challenge/leaderboard/

In [ ]:
train_data = pd.read_csv('hecktor-task2-TRAINIMAGES.csv', index_col='pid')
test_data = pd.read_csv('hecktor-task2-TESTIMAGES.csv')

In [ ]:
test_data

In [ ]:
train_data

In [ ]:
#Question about whether we use "correct" RFS: 
#At least we don't have any negative target values, so it doesn't make sense to multiply predictions with -1
train_data['RFS'].describe()

In [ ]:
#"Median RFS of 14 months in the training dataset", according to the Dataset page
#This is not true for our dataset, where I get approx. 40 months: 
# Divides with 30 because it is approx 30 days in a month (and RFS is measured in days)
1207/30

#Also, there are 488 observations in training and 359 in test set, while on the web page, it says 489 and 339, respectively. 
# https://hecktor.grand-challenge.org/Data/

### Data exploration

In [ ]:
test_data.isna().sum()

In [ ]:
train_data.isna().sum()

In [ ]:
# Tobacco, Alcohol and performance status are probably important for prediction of survival
#However, alcohol is missing for almost all patients, and should probably be excluded...
train_data = train_data.drop(['Alcohol'], axis=1)
test_data = test_data.drop(['Alcohol'], axis=1)

### Estimate the kidney function (using average serum creatinine values
#### https://www.mayoclinic.org/tests-procedures/creatinine-test/about/pac-20384646
#### Using micromoles/mL
#### Cockraft-Gault formula: https://gpnotebook.com/simplepage.cfm?ID=x20191025171810070107

In [ ]:
#Since serum creatinine is not available in dataset, we use average values for males and females
male_creatinine = (65.4 + 119.2)/2
female_creatinine = (52.2 + 91.9)/2

In [ ]:
#Estimate the kidney function in the training data
gfr_list=[]
for i in range(train_data.shape[0]):
    if train_data.iloc[i,5]==0:
        #print('This is a male:', train_data.iloc[i,:])
        temp_gfr = (((140 - train_data.iloc[i,6]) * train_data.iloc[i,7])*1.23)/male_creatinine
    else:
        #print('This is a female:', train_data.iloc[i,:])
        temp_gfr = 0.85*((((140 - train_data.iloc[i,6]) * train_data.iloc[i,7])*1.23)/female_creatinine)
    gfr_list.append(temp_gfr)

In [ ]:
#Repeat for the test set:
gfr_list_test=[]
for i in range(test_data.shape[0]):
    if test_data.iloc[i,5]==0:
        #print('This is a male:', train_data.iloc[i,:])
        temp_gfr = (((140 - test_data.iloc[i,6]) * test_data.iloc[i,7])*1.23)/male_creatinine
    else:
        #print('This is a female:', train_data.iloc[i,:])
        temp_gfr = 0.85*((((140 - test_data.iloc[i,6]) * test_data.iloc[i,7])*1.23)/female_creatinine)
    gfr_list_test.append(temp_gfr)

In [ ]:
train_data['eGFR'] = gfr_list
test_data['eGFR'] = gfr_list_test

In [ ]:
#Split data into X and y
y_train = train_data['RFS']
y_test = test_data['RFS']
X_train = train_data.drop(['RFS'], axis = 1)
X_test = test_data.drop(['RFS'], axis = 1)

In [ ]:
#Switch the axis to get the RFS as last column:
new_traindata = pd.concat([X_train, y_train], axis = 1)

In [ ]:
# Plot the correlations for the training data:
plt.figure(figsize=(10,10))
corr = new_traindata.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
heat_map = sns.heatmap(corr, mask = mask, annot = True)

plt.xticks(size = 14)
plt.yticks(size = 14)
plt.title('Correlations between input features and RFS', size=18)

plt.tight_layout()
plt.show()
#plt.savefig('Diagonal_HeatmapTrainingData_Task2_Annotated.png')

## Create an XGBoost regressor:

In [ ]:
#Hyperparameter search: 
params = {
    'n_estimators': [110,120,130],
    'learning_rate':[0.1,0.075,0.05],
    'max_depth':[5,4,3,2],
    'subsample':[0.8,0.75, 0.7],
    'colsample_bytree':[0.7, 0.6, 0.5],
    'colsample_bynode':[1,0.9],
    'colsample_bylevel':[0.9, 0.8, 0.7]
}

In [ ]:
model = XGBRegressor(booster = 'gbtree', random_state = 42, objective = 'reg:squarederror')
kfold = KFold(n_splits = 10, shuffle = True, random_state=42)

xgb_model = RandomizedSearchCV(
    estimator = model,
    param_distributions = params,
    random_state = 42,
    n_jobs = -1,
    cv = kfold,
    scoring = 'neg_mean_squared_error'
)
xgb_model.fit(X_train, y_train)

In [ ]:
print('Best score:',np.sqrt(-1*xgb_model.best_score_))
xgb_model.best_params_

In [ ]:
#Pick the most promising hyperparameter values and fit the model on the entire training dataset:
my_model = XGBRegressor(booster = 'gbtree', random_state = 42, objective = 'reg:squarederror',
                       subsample = 0.7, n_estimators = 120, max_depth = 4, learning_rate = 0.05, 
                       colsample_bytree = 0.6, colsample_bynode=1, colsample_bylevel=0.8)
my_model.fit(X_train, y_train)



In [ ]:
#Predict on the training data:
y_pred = my_model.predict(X_train)

In [ ]:
#Calculating RMSE on training data:
np.sqrt(mean_squared_error(y_pred, y_train))

## Predicting on the test set:

In [ ]:
test_predictions = my_model.predict(X_test)

In [ ]:
test_predictions

In [ ]:
#Concatenate X_test with the predicted values: 
X_test_predicted = pd.concat([X_test, pd.DataFrame(test_predictions, columns = ['Test_predictions'])], axis = 1)

In [ ]:
X_test_predicted['Test_predictions_rounded']= X_test_predicted['Test_predictions'].round(0)

In [ ]:
#Read in predictions for only the patients that should be included in Task 2: 
task2_predictions = pd.read_csv('hecktor-task2-andrea339.csv')
task2_predictions['OriginalPrediction'] = task2_predictions['-Prediction']*-1
task2_predictions['OriginalPrediction_rounded'] = task2_predictions['OriginalPrediction'].round(0)

In [ ]:
filtering_list = task2_predictions['OriginalPrediction_rounded'].tolist()

In [ ]:
#Select only the rows with predicted values for the 339 patients:
X_test_predicted = X_test_predicted[X_test_predicted['Test_predictions_rounded'].isin(filtering_list)]

In [ ]:
#Need to drop three more rows, since they do not belong to the test set:
X_test_predicted = X_test_predicted.drop([313, 336, 343], axis = 0)

In [ ]:
X_test_predicted.shape

In [ ]:
X_test_predicted

In [ ]:
#Remove columns not used by the model: 
y_test_predicted = X_test_predicted['Test_predictions']
X_test_predicted = X_test_predicted.drop(['Test_predictions', 'Test_predictions_rounded'], axis = 1)
X_test_predicted

## Investigating SHAP values (should also do this in R)

In [ ]:
import shap
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(my_model, X_train)
#explainer.feature_names = ['DOSE', 'TXT', 'TROUGH','1 HOUR', 'DIFFERENCE 1 HOUR','DIFFERENCE 3 HOURS', 'CONC_DIFFERENCE1','CONC_DIFFERENCE2','CONC_DIFFERENCE3','ON TIME','ESTIMATED 3 HOURS']
shap_values = explainer.shap_values(X_test_predicted.iloc[:,:])
shap.summary_plot(shap_values,X_test_predicted,plot_type="bar", max_display=X_test_predicted.shape[1])

### Customize the plot:

In [ ]:
#Get the global SHAP values for each feature
shap_list = []
for i in range(14):
    shap_list.append(abs(shap_values[:,i,]).mean())
    print(abs(shap_values[:,i,]).mean())

In [ ]:
feature_names = X_test.columns.tolist()
shap_dict = {}
for i in range(len(shap_list)):
    shap_dict[feature_names[i]] = shap_list[i]
    
#Sort from highest SHAP-value:
shap_dict = {k:v for k, v in sorted(shap_dict.items(), key = lambda item: item[1], reverse = True)}

In [ ]:
shap_dict

In [ ]:
most_imp_features = []
most_imp_values = []
for k,v in shap_dict.items():
    most_imp_features.append(k)
    most_imp_values.append(v)
#most_imp_features = most_imp_features[:10]
#most_imp_values = most_imp_values[:10]
#Comment out for standing plots:
most_imp_features.reverse()
most_imp_values.reverse()

In [ ]:
most_imp_features

In [ ]:
#plt.barh(most_imp_features, most_imp_values, color = '#D95B43')
plt.barh(most_imp_features, most_imp_values, color = '#542437')
ax = plt.gca()
plt.xlabel("SHAP value", size=14)
plt.xticks(size = 12)
plt.yticks(size = 12)
plt.draw()
plt.tight_layout()
fig = plt.gcf()
#Save as .svg for increased resolution
#fig.savefig("SHAP_XGBoost_Task2.pdf", format="pdf")

In [ ]:
#Save model: 
#my_model.save_model('hecktor_xgboostModel.model')